In [12]:
from collections import namedtuple
import os, sys
import urllib.request as request
from zipfile import ZipFile
from deepClassifier.constant import *
from deepClassifier.utils.common import read_yaml, create_directories

In [25]:
import os
os.chdir("D:\PycharmProjects\DS_ML_Self\FSDS_NOV_deepCNNClassifier")

In [26]:
os.getcwd()

'D:\\PycharmProjects\\DS_ML_Self\\FSDS_NOV_deepCNNClassifier'

In [14]:
DataIngestionConfig = namedtuple("DataIngestionConfig",[
    'root_dir',
    'source_URL',
    'local_data_file',
    'unzip_dir',
])

In [33]:
class ConfigurationManage:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH,):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config
        

In [16]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
    
    def _preprocess(self, zf:ZipFile, filepath:str, working_dir:str):
        target_filepath = os.path.join(working_dir, filepath)
        if not os.path.exists(target_filepath):
            zf.extract(filepath, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)
    
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file,mode='r') as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for file in updated_list_of_files:
                self._preprocess(zf, file, self.config.unzip_dir)
    
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ('Cat' in f or 'Dog' in f)]
            
    
    

In [17]:
from dataclasses import dataclass

@dataclass()
class DataIngestionConfig:
    root_dir: Path
    source_URL: str 
    local_data_file: Path
    unzip_dir: Path

In [30]:
path = Path('configs\config.yaml')
read_yaml(path)

[2022-09-18 14:07:54,308: INFO: common]: yaml file: configs\config.yaml loaded successfully


ConfigBox({'artifact_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}})

In [32]:
try:
    config = ConfigurationManage()
    data_ingestion_config = config.get_data_ingestion()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2022-09-18 14:08:49,262: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-09-18 14:08:49,266: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-18 14:08:49,271: INFO: common]: created directory at: artifacts
[2022-09-18 14:08:49,275: INFO: common]: created directory at: artifacts/data_ingestion
